# Create list of files from MAST download

Credit Jo Taylor.

In [ ]:
from astropy.io import fits
import pandas as pd
import glob

## Get list of files with header keywords

In [ ]:
df = pd.DataFrame(columns=["FILENAME", "TARG_RA", "TARG_DEC", "FILTER", "PUPIL", "PATT_NUM", "NUMDTHPT", "XOFFSET", "YOFFSET"])

for file in glob.glob("./mastDownload/JWST/jw02079004*/*rate.fits"):
    image = fits.open(file)
    
    df2 = pd.DataFrame({"FILENAME"   : [image[0].header["FILENAME"]],
                        "TARG_RA"     : [image[0].header["TARG_RA"]],
                        "TARG_DEC"     : [image[0].header["TARG_DEC"]],
                        "FILTER"     : [image[0].header["FILTER"]],
                        "PUPIL"      : [image[0].header["PUPIL"]],
                        "PATT_NUM"   : [image[0].header["PATT_NUM"]],
                        "NUMDTHPT"   : [image[0].header["NUMDTHPT"]],
                        "XOFFSET"    : [image[0].header["XOFFSET"]],
                        "YOFFSET"    : [image[0].header["YOFFSET"]]
                       })
    
    df = pd.concat([df, df2], ignore_index = True, axis = 0)
    
dfsort = df.sort_values('FILENAME', ignore_index = True)

## Save to csv file

In [ ]:
dfsort.to_csv("./list_ngdeep_rate.csv",sep=',')

## Move files to preferred location

At this point, I move the rate files and the association files to a preferred place on my machine:
- all rate files under NGDEEP/rate
- level 2 association files under NGDEEP/asn_level2
- level 3 association files under NGDEEP/asn_level3